In [ ]:
import pandas as pd
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
import janestreet
import torch.nn.functional as F
import torch
from tqdm import tqdm
import os
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

In [ ]:
def load_data(PATH):
    dt = pd.read_csv(PATH)
    dt = pd.DataFrame(dt)
    return dt

In [ ]:
class CustomDataset:
    def __init__(self, dataset, target):
        self.dataset = dataset
        self.target = target

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        return {
            'x': torch.tensor(self.dataset[item, :], dtype=torch.float),
            'y': torch.tensor(self.target[item, :], dtype=torch.float)
        }

In [ ]:
class Auto_encoder(torch.nn.Module):

    def __init__(self, number_features, std=0.05):
        super(Auto_encoder, self).__init__()

        # ENCODER
        self.linear_1 = torch.nn.Linear(number_features, num_hidden_1)

        # DECODER
        self.linear_2 = torch.nn.Linear(num_hidden_1, number_features)

    def encoder(self, x):
        encoded = self.linear_1(x)
        encoded = F.relu(encoded)
        return encoded

    def decoder(self, encoded_x):
        logits = self.linear_2(encoded_x)
        decoded = torch.sigmoid(logits)
        return decoded

    def forward(self, x):
        # ENCODER
        encoded = self.encoder(x)

        # DECODER
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
class MLP(torch.nn.Module):

    def __init__(self, number_features, number_classes):
        super(MLP, self).__init__()
        self.linear_1 = torch.nn.Linear(number_features, num_hidden_1, bias=True)
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2, bias=True)
        self.linear_3 = torch.nn.Linear(num_hidden_2, num_hidden_3, bias=True)
        self.linear_out = torch.nn.Linear(num_hidden_3, number_classes, bias=True)

    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        out = self.linear_3(out)
        out = F.relu(out)
        out = self.linear_out(out)
        out = torch.sigmoid(out)
        return out

In [ ]:
def training_auto_encoder(auto_encoder, number_epochs, save_pretrained=True):
    start_time = time.time()
    for epoch in range(number_epochs):
        auto_encoder.train()
        for iteration, batch in enumerate(train_loader):
            features = batch['x'].to(device)
            # FORWARD AND BACK PROP
            decoded = auto_encoder(features)
            cost = criterion_autoencoder(decoded, features)
            optimizer_autoencoder.zero_grad()
            cost.backward()

            # UPDATE MODEL PARAMETERS
            optimizer_autoencoder.step()

            # LOGGING
            if not iteration % 50:
                print('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                      % (epoch + 1, num_epochs, iteration, len(train_loader), cost))

    if save_pretrained:
        torch.save(auto_encoder.state_dict(), './auto_encoder.pt')
        print("-------------SAVED AUTO ENCODER PRETRAIN-------------")
    return auto_encoder

In [ ]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    threshold = 0.5
    with torch.no_grad():
        for batch in data_loader:
            features = batch['x'].to(device)
            targets = batch['y'].to(device)
            encoded = auto_encoder(features)
            features = torch.cat([encoded, features], 1)
            preds = net(features)
            preds = torch.where(preds > threshold, torch.tensor(1.).cuda(), torch.tensor(0.).cuda())
            num_examples += targets.shape[0]
            correct_pred += sum([1 if torch.equal(pred, target) else 0 for pred, target in zip(preds, targets)])
        return correct_pred / num_examples * 100

In [ ]:
def training_MLP(model, number_epochs, save_pretrained=True):
    for epoch in range(number_epochs):
        model.train()
        for iteration, batch in enumerate(train_loader):
            features = batch['x'].to(device)
            targets = batch['y'].to(device)
            targets = torch.squeeze(targets)
            encoded = auto_encoder(features)
            features = torch.cat([encoded, features], 1)
            preds = model(features)
            # FORWARD AND BACK PROP
            cost = criterion(preds, targets)
            optimizer.zero_grad()
            cost.backward()

            # UPDATE MODEL PARAMETERS
            optimizer.step()

            # LOGGING
            if not iteration % 50:
                print('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' % (epoch + 1, num_epochs, iteration, len(train_loader), cost))

        with torch.set_grad_enabled(False):
            print('Epoch: %03d/%03d training accuracy: %.2f%%' % (epoch + 1, num_epochs, compute_accuracy(model, validation_loader)))
            
    if save_pretrained:
        torch.save(model.state_dict(), './MLP.pt')
        print("-------------SAVED MLP PRETRAIN-------------")
    return model

In [ ]:
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if groups[idx] in group_dict:
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]

                train_array = np.sort(np.unique(
                    np.concatenate((train_array,
                                    train_array_tmp)),
                    axis=None), axis=None)

            train_end = train_array.size

            for test_group_idx in unique_groups[group_test_start: group_test_start + group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                    np.concatenate((test_array,
                                    test_array_tmp)),
                    axis=None), axis=None)

            test_array = test_array[group_gap:]

            if self.verbose > 0:
                pass

            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
threshold = 0.5
def predict(auto_encoder, model):
    env = janestreet.make_env()
    env_iter = env.iter_test()
    for (test_df, pred_df) in tqdm(env_iter):
        if test_df['weight'].item() > 0:
            test_df = pd.DataFrame(test_df, columns=column_features)
            test_df = test_df.fillna(method='ffill').fillna(method='bfill')
            X_test = torch.FloatTensor(np.array(test_df)).to(device)
            encoded = auto_encoder(X_test)
            X_test = torch.cat([encoded, X_test], 1)
            preds = model(X_test)
            preds = preds.cpu().detach().numpy()
            preds = np.prod(preds, 1)
            action = np.where(preds > threshold, 1, 0).astype(int)
            pred_df.action = action
        else:
            pred_df.action = 0
        env.predict(pred_df)

In [ ]:
if __name__ == "__main__":
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    train_path = "/kaggle/input/jane-street-market-prediction/train.csv"
    data = load_data(train_path)

    random_seed = 1
    learning_rate = 0.05
    num_epochs = 20
    batch_size = 4096

    num_hidden_1 = 128
    num_hidden_2 = 64
    num_hidden_3 = 32
    num_classes = 5
    FOLDS = 2

    gkf = PurgedGroupTimeSeriesSplit(n_splits=FOLDS, group_gap=20)
    data = data.query('date > 85').reset_index(drop=True)
    data = data.astype({c: np.float32 for c in data.select_dtypes(include='float64').columns})
    data = data.fillna(method='ffill').fillna(method='bfill')
    data = data.query('weight > 0').reset_index(drop=True)
    resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
#     data['action'] = (data['resp'] > 0).astype('int')
    y = np.stack([(data[c] > 0).astype('int') for c in resp_cols]).T
    data.drop(columns=resp_cols, inplace=True)
    splits = list(gkf.split(y, groups=data['date'].values))
    column_features = [c for c in data.columns if 'feature' in c]
    num_features = len(column_features)
    X = data[column_features].values
    torch.manual_seed(random_seed)
    
    model = MLP(number_features=num_features*2, number_classes=num_classes)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()
    
    auto_encoder = Auto_encoder(number_features=num_features)
    auto_encoder = auto_encoder.to(device)
    criterion_autoencoder = nn.MSELoss()
    optimizer_autoencoder = torch.optim.Adam(auto_encoder.parameters(), lr=0.001)
    
    start_time = time.time()
    for fold, (train_indices, valid_indices) in enumerate(splits):
        X_train, X_valid = X[train_indices], X[valid_indices]
        y_train, y_valid = y[train_indices], y[valid_indices]
        train_data, train_target = X_train, y_train
        validation_data, validation_target = X_valid, y_valid
        train_dataset = CustomDataset(dataset=train_data, target=train_target)
        validation_dataset = CustomDataset(dataset=validation_data, target=validation_target)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
        auto_encoder = training_auto_encoder(auto_encoder, num_epochs)
        model = training_MLP(model, num_epochs)
    print('Total Training Time: %.2f min' % ((time.time() - start_time) / 60))
    path_MLP = './MLP.pt'
    path_auto_encoder = './auto_encoder.pt'
    if os.path.exists(path_auto_encoder):
        auto_encoder = Auto_encoder(number_features=num_features)
        auto_encoder.load_state_dict(torch.load(path_auto_encoder))
        auto_encoder.to(device)
        auto_encoder.eval()
    if os.path.exists(path_MLP):
        model = MLP(number_features=num_features*2, number_classes=num_classes)
        model.load_state_dict(torch.load(path_MLP))
        model.to(device)
        model.eval()
    predict(auto_encoder, model)